In [1]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
from scipy.stats import norm, multivariate_normal
import itertools
from itertools import combinations_with_replacement, combinations, permutations
import copy
import Tasmanian
import matplotlib.pyplot as plt

import bo_methods_lib
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pickle
import gzip

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import integrate

import warnings
warnings.simplefilter("ignore", category=RuntimeWarning)
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-02 10:32:35.624574: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-02 10:32:35.624628: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-02 10:32:35.626502: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-02 10:32:35.635659: I tensorflow/core/platform/cpu_feature_guard

In [2]:
def func1(x):
    # for 2D: f(x)= 1 - x1^2 - x2^2
    return 1 - x[0]**2 - x[1]**2

def func2(x):
    return np.maximum(func1(x), 0)

def func3(x):
    return 0.5*(np.sqrt(func1(x)**2 + 1e-5) + func1(x))

def func4(x):
    return np.sin(x[0])*x[1]

def func5(x):
    return x[0]*x[1]

#Create test functions*pdf(normal)
def get_mean_covar(vars_shape, mean = False, covar = False):
    np.random.seed = 1
    if mean is True:
        mean = np.array([1.0,1.2])#np.ones(vars.shape[1])
    else:
        mean = np.zeros(vars_shape)
    if covar is True:
        B = np.ones((vars_shape,vars_shape))*0.5
        covar = np.dot(B.T, B) + 0.5*np.eye(vars_shape)
    else:
        covar = np.eye(vars_shape)

    return mean, covar

#Create test functions*pdf(normal)
def func1_pdf(x1, x2, mean = True, covar = True):
    # for 2D: f(x)= 1 - x1^2 - x2^2
    vars = np.array([x1,x2]).reshape(-1,1)
    np.random.seed = 1
    mean, covar = get_mean_covar(len(vars), mean = mean, covar = covar)
    pdf = multivariate_normal.pdf(vars.T, mean=mean, cov=covar)
    return func1(vars)*pdf

def func2_pdf(x1, x2, mean = True, covar = True):
    # for 2D: f(x)= 1 - x1^2 - x2^2
    vars = np.array([x1,x2]).reshape(-1,1)
    np.random.seed = 1
    mean, covar = get_mean_covar(len(vars), mean = mean, covar = covar)
    pdf = multivariate_normal.pdf(vars.T, mean=mean, cov=covar)
    return func2(vars)*pdf

def func3_pdf(x1, x2, mean = True, covar = True):
    # for 2D: f(x)= 1 - x1^2 - x2^2
    vars = np.array([x1,x2]).reshape(-1,1)
    np.random.seed = 1
    mean, covar = get_mean_covar(len(vars), mean = mean, covar = covar)
    pdf = multivariate_normal.pdf(vars.T, mean=mean, cov=covar)
    return func3(vars)*pdf

def func4_pdf(x1, x2, mean = True, covar = True):
    # for 2D: f(x)= 1 - x1^2 - x2^2
    vars = np.array([x1,x2]).reshape(-1,1)
    np.random.seed = 1
    mean, covar = get_mean_covar(len(vars), mean = mean, covar = covar)
    pdf = multivariate_normal.pdf(vars.T, mean=mean, cov=covar)
    return func4(vars)*pdf

def func5_pdf(x1, x2, mean = True, covar = True):
    # for 2D: f(x)= 1 - x1^2 - x2^2
    vars = np.array([x1,x2]).reshape(-1,1)
    np.random.seed = 1
    mean, covar = get_mean_covar(len(vars), mean = mean, covar = covar)
    pdf = multivariate_normal.pdf(vars.T, mean=mean, cov=covar)
    return func5(vars)*pdf

In [3]:
#Test MC vs Sparse Grid on a fxn*norm.pdf w/ gaussian random variable
def plot_sg_points(points_p, func):
    # Define the range for the input space (for visualization)
    min = np.min(points_p, axis=0)
    max = np.max(points_p, axis=0)
    x_range = np.linspace(min[0], max[0], 100)
    y_range = np.linspace(min[1], max[1], 100)
    X, Y = np.meshgrid(x_range, y_range)

    # Evaluate the function over the grid
    Z = func([X, Y])

    # Plotting the function surface and points
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Plot the continuous function surface
    ax.plot_surface(X, Y, Z, alpha=0.5, cmap='viridis')

    # Scatter plot of the points from Tasmanian grid
    values = func(points_p.T)
    ax.scatter(points_p[:, 0], points_p[:, 1], values, c='r', marker='o')

    # Labeling the plot
    ax.set_title('Num Points =' + str(len(points_p)), fontsize=20)
    ax.set_xlabel(r"$ϵ_1$", fontsize=20)
    ax.set_ylabel(r"$ϵ_2$", fontsize=20)
    ax.set_zlabel(r"$f(ϵ)$", fontsize=20)

    # Show plot
    plt.show()

def sg_integ_gauss(func, dim, gp_mean, gp_covar, depth =5, seed = 1, verbose = False):
    np.random.seed = seed
    #Get grid points and weights
    grid_p = Tasmanian.SparseGrid()
    grid_p.makeGlobalGrid(dim, 0, depth, "hyperbolic", 'gauss-hermite-odd')
    points_p_old = grid_p.getPoints()*np.sqrt(2)
    weights_p = grid_p.getQuadratureWeights()
            
    #Diagonalize covariance matrix
    L = scipy.linalg.cholesky(np.real(gp_covar), lower=True)
    transformed_points = L@points_p_old.T
    points_p = gp_mean[:, np.newaxis].T + transformed_points.T

    if verbose:
        plot_sg_points(points_p, func)

    #Evaluate function
    y = func(points_p.T)

    integ = (np.pi**(-dim/2))*y@weights_p.T
    return integ

def mc_integ_gauss(func, dim, n, gp_mean, gp_covar, seed = 1):
    # Monte Carlo integration of given function over domain from a to b (for each parameter)
    # dim: dimensions of function
    #Initialize total ei  
    # n = math.ceil((norm.ppf(1-0.05/2)/0.05)**2)  
    # np.random.seed(seed)

    #Get random variable from normal distribution
    x_list = np.random.multivariate_normal(gp_mean, gp_covar, n)

    #Evaluate function
    y = func(x_list.T)

    y_mean =  np.mean(y)

    integ = y_mean

    return integ

In [4]:
#Test MC vs Sparse Grid on a fxn*norm.pdf w/ gaussian random variable
column_names = ["scipy", "mc n=200", "mc n=2000", "mc n=20000", "sg depth=5", "sg depth=10", "sg depth=20"]
column_names = ["sg depth=5", "sg depth=10", "sg depth=20"]
results_df = pd.DataFrame(columns=column_names)
func_list = [func1, func2, func3, func4, func5] #, func5, func6, func7, func8, func9]
func_list2 = [func1_pdf,  func2_pdf, func3_pdf, func4_pdf, func5_pdf]

func_list = [func1, func2, func3] #, func5, func6, func7, func8, func9]
func_list2 = [func1_pdf, func2_pdf, func3_pdf]

seed = 1
depth1 = 5
depth2 = 10
depth3 = 20
n1 = 200
n2 = 2000
n3 = 20000
add_mean = True
add_var = True

for i in range(len(func_list)):
    func = func_list[i]
    func_pdf = func_list2[i]

    if i > 5:
        #Scipy intractable for 5D problems
        z_scipy = "n/a"

        dim =5
    else:
        # z_scipy = integrate.dblquad(func_pdf, -np.inf, np.inf, -np.inf, np.inf, args = (add_mean, add_var))[0]
        dim =2

    mean, covar = get_mean_covar(dim, mean=add_mean, covar = add_var)
    if i == 0:
        print("Mean:", mean)
        print("Covar:", covar)
    #Evaluate "improvement (the normal function)" integral using MC
    # z_mc1 = mc_integ_gauss(func, dim, n1, mean, covar)
    # z_mc2 = mc_integ_gauss(func, dim, n2, mean, covar)
    # z_mc3 = mc_integ_gauss(func, dim, n3, mean, covar)
    #Evaluate "improvement (the normal function)" integral using sparse grid
    z_sg1 = sg_integ_gauss(func, dim, mean, covar, depth1)
    z_sg2 = sg_integ_gauss(func, dim, mean, covar, depth2)
    z_sg3 = sg_integ_gauss(func, dim, mean, covar, depth3)

    #Add to dataframe results
    iter_df = pd.DataFrame(columns=column_names)
    # integ_results = [z_scipy, z_mc1, z_mc2, z_mc3, z_sg1, z_sg2, z_sg3]
    integ_results = [z_sg1, z_sg2, z_sg3]
    # Add the new row to the DataFrame
    iter_df.loc[0] = integ_results
    results_df = pd.concat([results_df.astype(iter_df.dtypes), iter_df], ignore_index=True)

Mean: [1.  1.2]
Covar: [[1.  0.5]
 [0.5 1. ]]


In [5]:
results_df

,sg depth=5,sg depth=10,sg depth=20
0,-3.440000,-3.440000,-3.440000
1,0.108712,0.079824,0.087148
2,0.108718,0.079894,0.087122


In [6]:
# results_df.to_csv("mc_test_gauss_all_hyper.csv")